In [1]:
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random, operator

In [2]:
class IterRegistry(type):
    def __iter__(cls):
        return iter(cls._registry)

In [3]:
class Firm:
    #class attributes intialisation, to be updated with the addition of each instance
    _registry = []
    
    def __init__(self, index, capacity, number_of_vessel, operational_speed, design_speed, 
                 min_speed, max_speed, main_engine_power, aux_engine_power,
                 fix_cost, fuel_type):
        '''__init__ a method to describe the poperty that all shipping firm have'''
        self._registry.append(self)
        self.index = index
        self.capacity = capacity # Vessel capacity in TEU per firm, 
        self.number_of_vessel = number_of_vessel # # of vessels per firms
        self.operational_speed = operational_speed
        self.design_speed = design_speed  #design speed of the vessel in knots
        self.min_speed = min_speed #min vessel speed
        self.max_speed = max_speed #max vessel speed         
        self.main_engine_power =  main_engine_power  # PS_m : main engine power in kW
        self.aux_engine_power =  aux_engine_power  # auxiliary engine power [kW] 
        self.fix_cost = fix_cost #Daily cost of vessel (USD/Day) $25,000
        self.fuel_type = fuel_type
        #self.bau_emission = bau_emission
         
   
    # Firm methods 
    def get_market_share(self , market_instance):
        '''return market share of the firm accoding to it s capacity'''
        self.transport_capacity = np.multiply(self.number_of_vessel , self.capacity)
        self.market_share= np.divide(self.transport_capacity, market_instance.market_capacity)
        return self.market_share
        
    def get_firm_param(self, market_instance):
        self.psy = np.multiply (self.market_share , market_instance.market_psy)
        return self.psy
    
    def get_firm_demand(self, market_instance):
        self.firm_demand = np.multiply (self.market_share , market_instance.market_demand)
        return self.firm_demand 
    
    def get_min_number_of_vessel(self, market_instance):
        self.time_at_sea = np.true_divide(market_instance.distance,  self.operational_speed)
        self.voyage_time = self.time_at_sea  + market_instance.port_time 
        self.number_of_trips_to_meet_demand = np.true_divide(self.firm_demand, self.capacity)
        self.max_trips_per_vessel = np.true_divide(market_instance.annual_working_time, self.voyage_time)
        self.min_number_of_vessel_to_meet_demand = np.true_divide(self.number_of_trips_to_meet_demand, self.max_trips_per_vessel)
        return self.min_number_of_vessel_to_meet_demand , self.number_of_trips_to_meet_demand, self.time_at_sea 
            
    def get_ship_energy_efficiency(self, market_instance):
        self.main_fuel_parameter = market_instance.SFOC_main * market_instance.eng_load_main * self.main_engine_power * 10**(-6)
        self.ship_energy = np.multiply(self.main_fuel_parameter, np.power(self.design_speed, -3) )      
        return self.ship_energy
        
    def get_main_fuel_cons(self, market_instance):
        self.main_fuel_cons = market_instance.distance * self.ship_energy * self.operational_speed**2 *self.number_of_trips_to_meet_demand
        return self.main_fuel_cons
        
    def get_aux_fuel_cons(self, market_instance): 
        self.aux_fuel_parameter = market_instance.SFOC_aux * market_instance.eng_load_aux * self.aux_engine_power * 10**(-6)
        self.aux_fuel_cons =  self.aux_fuel_parameter * self.time_at_sea 
        return self.aux_fuel_cons 
        
    def get_fuel_cost(self, market_instance):
        
        pass
    
    def get_total_cost(self):
        pass
    
    def get_revenue(self):
        pass
    
    def get_period_profits(self):
        pass
    
    def get_discounted_period_profit(self):
        pass
    
    
    def get_firm_emission(self,firm_id,fuel_choice,fuel_consumed,factor):
        pass
            
    def get_firm_abatement_benefits():
        pass

In [4]:
class MarketConfig: 
    """Market Configuration and simulation game set up
    .. version: : 0.1
    
    Parameters
    ----------
    
    Attributes
    ----------
    
    """
    #class attributes
    t0 = 2016 #The starting year for evaluation of the pay-offs
    T = 35  # Planning Horizon t = 2016,...,2040 # 100 year scope 
    year = np.arange(2016, 2051)
    distance = 11810 #nautical miles #Notteboom (2006)  
    port_time = 264 #hours ==> 11 days in a year  #Notteboom (2006)
    demand_16_20 = np.array([1303780  for j in range(5)]) #in TEU #placehplder to be calibrated with the chosen route 
    annual_working_time = 6480 #hours per year, assumption   #hours per year, assumption 
    initial_freight_rate = 1800 ## in US$/TEU 
    SFOC_main = 206 #g/kWh, specific daily main engine fuel oil consumption rate
    SFOC_aux = 221 # specific fuel oil consumption of the auxiliary engine [g/kW h], 
    eng_load_main = 0.8 # % 
    eng_load_aux = 0.5 # engine load of the auxiliary engine [\%]


    def __init__(self, number_of_firms, demand_income_elasticity, demand_price_elasticity, 
                 freight_rate, fuel_data, discount_rate ):
        '''__init__ a method to describe the poperty that the shipping market and simulation game has'''
        self.number_of_firms = number_of_firms
        self.demand_income_elasticity = demand_income_elasticity #Constant income elasticity #IMF
        self.demand_price_elasticity =  demand_price_elasticity #Constant own price elasticity#IMF
        self.freight_rate = freight_rate ## in US$/TEU
        self.fuel_data = fuel_data 
        self.discount_rate = discount_rate 
        #self.bau_industry_emission =  bau_industry_emission 
        #self.bau_pollution_stock = bau_pollution_stock
        self.market_capacity = 0 
        self.pollution_stock = 0
        self.industry_damage = 0
        self.global_abatement_benefits = 0
        
    
    def get_market_capacity (self, firm_instance):
        self.market_capacity += firm_instance.number_of_vessel * firm_instance.capacity
        return self.market_capacity 
    
    def get_freight_rate_ratio(self): 
        self.beta = np.divide(self.freight_rate, MarketConfig.initial_freight_rate) 
        return self.beta
   
    def get_market_demand(self):
        '''compute market level demand'''
        #---------> 1.Import real GDP growth data & compute GDP ratio : Source IMF@2020 #
        G_df = pd.read_csv('./data/real_growth_rate.csv') #import IMF data
        #-------> 2.Construct GDP projection path 2016-2050 based on projection growth data 
        gdp_growth= G_df.values[:,1:] 
        g =1 +(gdp_growth/100)  
        self.gdp = np.array([100.00 for j in range(MarketConfig.T)])
        for foo in range (1,MarketConfig.T):
            self.gdp[foo] = g[:,foo]* self.gdp[foo-1]
        #print(self.gdp)
        #--------->  "compute GDP ratio based on IMF@2020"
        gdp_ratio = np.array([1.00 for j in range(MarketConfig.T)])
        for moo in range (1,MarketConfig.T):
            sub_g = g[:,0:moo+1]
            #print(sub_g)
            gdp_ratio[moo] = np.prod(sub_g)
        
        #--------->  "compute freight rate ratio"
        self.freight_rate_ratio_multiplied = np.power(self.beta, self.demand_price_elasticity)
        #print(freight_rate_ratio_multiplied)
        
        #---------> "Project Transport Demand (industry demand)"
        loo =np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), self.freight_rate_ratio_multiplied ) #Will need to be updated in case fuel prices are varied over time  
        self.market_demand = np.multiply(loo, MarketConfig.demand_16_20[0])
        #print(Y) # size = 1 dimesion array with size T=35 years
        self.market_psy = np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), MarketConfig.demand_16_20[0] )
        return self.market_demand, self.gdp, self.market_psy

    
    
    def get_industry_emission(self):
        pass
    
    def get_pollution_stock(self):
        pass
    
    def get_damage(self):
        pass
    
    def get_global_abatement_benefits(self):
        pass 



In [5]:

#-------------------------------------------------- MAIN ------------------------------------------------------------- #

####################################### 1. Firm & Market specific Parameters ###########################################

                    #===========================>  Firm 1 Attributes <==========================#
index_1 = 1
capacity_1 = 5905
number_of_vessel_1= 50
intial_operational_speed_1 = 25 
design_speed_1 = 23.3 #design speed of the vessel in knots
min_speed_1 = 8.5
max_speed_1 = 30
main_engine_power_1 = 41186 # main engine power in kW
aux_engine_power_1 = 2433 # auxiliary engine power [kW] 
fix_cost_1 =  6750000  #Daily cost of vessel (USD/Day) $25,000
fuel_type_1 = "HFO"

                    #===========================>  Firm 2 Attributes <==========================#
index_2 = 2
capacity_2 = 6470
number_of_vessel_2 = 50
intial_operational_speed_2 = 2
design_speed_2 = 24.7 #design speed of the vessel in knots
min_speed_2 = 8.5
max_speed_2 = 30
main_engine_power_2 = 56273 # main engine power in kW
aux_engine_power_2 =  2433 # auxiliary engine power [kW] 
fix_cost_2 = 6750000 #Daily cost of vessel (USD/Day) $25,000
fuel_type_2 = "HFO"


                    #===========================> Market Attributes <==========================#
firms = 2
income_elasticity = 0.8 #Constant income elasticity #IMF
price_elasticity = -0.7 #Constant own price elasticity#IMF
freight_rate = 1800 ## in US$/TEU
fuel_data = pd.DataFrame(np.array([[422.50, 525.50, 597.00], 
                                [3.114,3.206,3.206],
                                [0.07,0.01,0.002]]),
                         columns=['HFO', 'ULSFO', 'MGO'],
                         index = ['price', 'carbon_factor', 'sulfur_factor'])
discount = 0.02  ##3  to 5 % transport canada ,#to be updated using the Ramsey rule,#pure rate of time preference of 1.5% + growth rate of consumption g * rate of risk conversion , an elasticity value of 2

In [6]:
""" This cell needs to run once throughout the kernel"""
####################################### 2.Simulation Game Initialisation ###########################################
firm_1 = Firm(index_1, capacity_1, number_of_vessel_1, intial_operational_speed_1, design_speed_1, 
                 min_speed_1, max_speed_1, main_engine_power_1, aux_engine_power_1,
                 fix_cost_1, fuel_type_1) 

firm_2 = Firm(index_2, capacity_2, number_of_vessel_2, intial_operational_speed_2, design_speed_2, 
                 min_speed_2, max_speed_2, main_engine_power_2, aux_engine_power_2,
                 fix_cost_2, fuel_type_2) 

game_config = MarketConfig(firms, income_elasticity, price_elasticity, freight_rate, 
                           fuel_data, discount)

####################################### 3.Get market capacity #################################
for  firm_object in Firm._registry:
    market_capacity_sim = game_config.get_market_capacity(firm_object)

In [7]:
####################################### 4.Get freight ratio for passethrough###########################
beta_sim = game_config.get_freight_rate_ratio()

In [8]:
####################################### 5.Get market demand #################################
market_demand_sim, gdp_sim, market_psy_sim = game_config.get_market_demand()
#print(market_demand_sim)

In [9]:
####################################### 6.Get firm market share #################################
firm_1_market_share_sim = firm_1.get_market_share(game_config)
firm_2_market_share_sim = firm_2.get_market_share(game_config)

In [10]:
####################################### 7.Get firm passthrough parameter #################################
firm_1_psy_sim = firm_1.get_firm_param(game_config)
firm_2_psy_sim = firm_2.get_firm_param(game_config)

In [11]:
####################################### 7.Get firm level demand #################################
firm_1_demand_sim = firm_1.get_firm_demand(game_config)
firm_2_demand_sim = firm_2.get_firm_demand(game_config)

In [12]:
####################################### 8.Get min # of vessels to meet demand #################################
firm_1_min_number_vessels_sim, firm_1_trips_to_meet_demand_sim, firm_1_time_at_sea_sim  = firm_1.get_min_number_of_vessel(game_config)
firm_2_min_number_vessels_sim, firm_2_trips_to_meet_demand_sim, firm_2_time_at_sea_sim= firm_2.get_min_number_of_vessel(game_config)

In [13]:
####################################### 9.Get ship energy efficiency #################################
firm_1_ship_energy_sim = firm_1.get_ship_energy_efficiency(game_config)
firm_2_ship_energy_sim = firm_2.get_ship_energy_efficiency(game_config)

####################################### 10.Get main engine fuel consumption #################################
firm_1_main_fuel_con_sim = firm_1.get_main_fuel_cons(game_config)
firm_2_main_fuel_con_sim = firm_2.get_main_fuel_cons(game_config)

####################################### 11.Get Auxi engine fuel consumption #################################
firm_1_aux_fuel_con_sim = firm_1.get_aux_fuel_cons(game_config)
firm_2_aux_fuel_con_sim = firm_2.get_aux_fuel_cons(game_config)

####################################### 12.Get Fuel engine fuel consumption #################################



In [14]:
####################################### 13.Get Firm Cost #################################




In [15]:
####################################### 13.Get Firm Revenue #################################


In [16]:
####################################### 14.Get Emissions #################################
